정형데이터 : 데이터베이스에 저장할수 있는 데이터, 행과 열 구분 ex)DB, 엑셀, csv

비정형데이터 : 텍스트, 오디오파일, 이미지, 영상 ->딥러닝 ->데이터 전처리 까다로움 <표현학습>

트리의 앙상블 : 여러개의 모델을 합하여 하나의 결과를 만들어 냄

트리는 과대적합하려는 경향이 있다 트리를 기반으로 여러개 만들어서 과대적합을 줄여 성능이 좋은 모델 만듬

랜덤포레스트: 트리를 랜덤으로 여러개 만드는 것

랜덤포레스트 훈련 방법

훈련세트 -> 랜덤샘플링(중복을 허용한 샘플링) ->부트스트랩샘플 -> 결정트리훈련 ->확률총합/트리개수



In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

#와인데이터불러오기
wine = pd.read_csv('https://bit.ly/wine-date')

data = wine[['alcohol', 'sugar', 'pH']].to_numpy()#데이터
target = wine['class'].to_numpy()#타깃

train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)#훈랸,테스트데이터구분

In [ ]:
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier#회귀의 경우 RandomForestRegressor
#RandomForestClassifier 전체특성의 개수에 루트값만큼 특성 선택
#100개 결정트리 사용
# RandomForestRegressor() : 전체 특성 사용.

rf = RandomForestClassifier(n_jobs=-1, random_state=42)#객체만들기
scores = cross_validate(rf, train_input, train_target, return_train_score=True, n_jobs=-1)
#객체,훈련데이터,타겟데이터,훈련세트점수반환,모든코어사용

print(np.mean(scores['train_score']), np.mean(scores['test_score']))
#훈련세트점수, 검증세트점수

0.9973541965122431 0.8905151032797809


In [ ]:
rf.fit(train_input, train_target)#훈련
print(rf.feature_importances_)#특성중요도->골고루 특성사용

[0.23167441 0.50039841 0.26792718]


In [ ]:
rf = RandomForestClassifier(oob_score=True, n_jobs=-1, random_state=42)
#oob:중복을 하여 데이터 샘플링 뽑히지 않는 샘플이 있다.. ->검증세트의 점수를 형가
#하듯이 사용하는 것이 가능
rf.fit(train_input, train_target)
print(rf.oob_score_)#검증세트와 점수차이가 거의 나지 않는다

0.8934000384837406


엑스트라트리: 100개의 트리 사용

부트스트랩 샘플사용X

성능억제->랜덤하게 분할-> 불순도 best찾고 노드분할

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier

et = ExtraTreesClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(et, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9974503966084433 0.8887848893166506


In [ ]:
et.fit(train_input, train_target)
print(et.feature_importances_)

[0.20183568 0.52242907 0.27573525]


랜덤포레스트 : 부트스트랩샘플링 사용->트리성능 억제

엑스트라트리 : 노드분할을 무작위로 함-> 트리성능 억제

그레이디언트 부스팅 : 손실함수값이 낮아지도록 회귀트리를 추가

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

gb = GradientBoostingClassifier(random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.8881086892152563 0.8720430147331015


In [ ]:
gb = GradientBoostingClassifier(n_estimators=500, learning_rate=0.2, random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)
#트리의개수:500, 기본100
#learning_rate(학습률):0.2, 기본:0.1

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9464595437171814 0.8780082549788999


In [ ]:
gb.fit(train_input, train_target)
print(gb.feature_importances_)

[0.15872278 0.68010884 0.16116839]


히스토그램 기반 그레이디언트부스팅

255개구간, 1개는 누락된 값 할당(결측치 무시)

In [ ]:
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier

hgb = HistGradientBoostingClassifier(random_state=42)
scores = cross_validate(hgb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9321723946453317 0.8801241948619236


In [ ]:
hgb.fit(train_input, train_target)
print(rf.feature_importances_)

[0.23167441 0.50039841 0.26792718]


In [ ]:
hgb.score(test_input, test_target)

0.8723076923076923

Permutation Importance(치환중요도) ->각각의 열의 특성을 섞음 원래데이터랑 성능 비교 값의 차이를 통해 특성의 중요도를 알 수 있다

In [ ]:
from sklearn.inspection import permutation_importance
hgb.fit(train_input, train_target)
result = permutation_importance(hgb, train_input, train_target, n_repeats=10,
                                random_state=42, n_jobs=-1)
#각 특성마다 10번씩 섞음

In [ ]:
print(result.importances_mean)
#정확도의 손실을 의미함

[0.08876275 0.23438522 0.08027708]


In [ ]:
result = permutation_importance(hgb, test_input, test_target, n_repeats=10,
                                random_state=42, n_jobs=-1)
print(result.importances_mean)
#테스트세트

[0.05969231 0.20238462 0.049     ]


In [ ]:
hgb.score(test_input, test_target)

0.8723076923076923

XGBoost

In [ ]:
from xgboost import XGBClassifier

xgb = XGBClassifier(tree_method='hist', random_state=42)
scores = cross_validate(xgb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.8824322471423747 0.8726214185237284


LightGBM:히스토그램기반

In [ ]:
from lightgbm import LGBMClassifier

lgb = LGBMClassifier(random_state=42)
scores = cross_validate(lgb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9338079582727165 0.8789710890649293


앙상블 보고서 (작성자:혼공머신)

앙상블학습은 정형데이터에서 가장 뛰어난 성능을 내는 머신러닝 알고리즘중 하나입니다.

1.랜덤포레스트: 부트스트랩 샘플사용. 대표적인 학습 알고리즘

2.엑스트라 트리: 결정트리의 노드를 랜덤하게 분할함

3.그레이디언트 부스팅: 이전 트리의 손실을 보완하는 식으로 결정트리를 연속해서 추가함

4.히스토그램기반 그레이디언트부스팅: 훈련데이터를 256개의 정수구간으로 나누어 빠르고 높은 성능을 냄

5.XGBoost, LightGBM